#### 자전거도로 제안
- 연도별 대여이력 정보를 불러와서 대여대여소, 반납대여소 두 값을 키로 보고 이용거리, 이용시간을 더하고, 갯수를 카운트하여 새로운 dataframe을 생성하여 저장
- 시각화

In [ ]:
# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기
import ExcelFile_Merge as em
import ExcelFile_Read as er

import pandas as pd

In [ ]:
station_info = em.Read_File(r'G:\내 드라이브\DataSet\_파킷 파일\서울시 공공자전거 대여소 정보\공공자전거 대여소 정보_통합본_최종.parquet')

df = em.Read_File(r'G:\내 드라이브\DataSet\_파킷 파일\서울특별시 공공자전거 대여이력 정보\gis\공공자전거 대여반납 그룹_2021_2024_출퇴근.parquet')
display(df.head(1))
df = pd.merge(df, station_info[['대여소번호', '위도', '경도']], left_on='대여대여소번호', right_on='대여소번호', how='left').rename(columns={'위도': '대여대여소위도', '경도': '대여대여소경도'})
df.drop('대여소번호', axis=1, inplace=True)
df = pd.merge(df, station_info[['대여소번호', '위도', '경도']], left_on='반납대여소번호', right_on='대여소번호', how='left').rename(columns={'위도': '반납대여소위도', '경도': '반납대여소경도'})
df.drop('대여소번호', axis=1, inplace=True)
display(df.head(1))
df = df[df['대여대여소번호'] != df['반납대여소번호']]
# display(df.isnull().sum())
# display(df[df['대여대여소명'].isnull()]['대여대여소번호'].unique())
# display(df[df['대여대여소번호'] == '상암센터 정비실'])
# display(df[df['대여대여소번호'] == '0중랑센터'])
grouped_df = df.groupby(['대여대여소번호', '반납대여소번호']).agg({
    '이용시간(분)': 'sum',
    '이용거리(M)': 'sum',
    '이용량': 'sum',
    '대여대여소명': 'last',
    '대여상세주소': 'last',
    '반납대여소명': 'last',
    '반납상세주소': 'last',
    '대여대여소위도': 'last',
    '대여대여소경도': 'last',
    '반납대여소위도': 'last',
    '반납대여소경도': 'last'
    }).reset_index()
display(grouped_df.head(1))

In [ ]:
import folium
import pandas as pd

# 예시 데이터프레임 준비: 출발지 위도, 출발지 경도, 도착지 위도, 도착지 경도, 가중치(사용량)
grouped_df = grouped_df.dropna()
grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)

# 데이터프레임 생성
data = {
    'start_lat': grouped_df['대여대여소위도'],
    'start_lon': grouped_df['대여대여소경도'],
    'end_lat': grouped_df['반납대여소위도'],
    'end_lon': grouped_df['반납대여소경도'],
    'usage': grouped_df['이용량']  # 가중치
}
df = pd.DataFrame(data)

# Min-Max Scaling 적용
df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
df = df.sort_values(by='usage_scaled', ascending=True)

# Folium 지도 생성 (서울을 중심으로 설정)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# 출발지-도착지 선 추가
for _, row in df.iterrows():
    # 좌표 리스트 정의
    locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
    
    # 가중치에 따른 밝기 및 선의 두께와 색상, 투명도 설정
    weight = 1 + 10 * row['usage_scaled']  # 기본 두께 1, 최대 두께 11
    red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 사용량에 비례
    blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 사용량에 반비례
    color = f'#{red_intensity:02x}55{blue_intensity:02x}'  # 초록빛을 더하여 그라데이션 효과
    
    # 투명도 설정
    opacity = 0.2 + 0.6 * row['usage_scaled']  # 적은 이용량은 투명도 0.2, 많은 이용량은 0.8

    # PolyLine 추가 (부드러운 선과 그라데이션)
    folium.PolyLine(
        locations,
        color=color,
        weight=weight,
        opacity=opacity,
        smooth_factor=2.0,  # 부드럽게 선을 표현
        tooltip=f'Usage: {row["usage"]}'
    ).add_to(m)

# 결과 지도 저장
m.save("bike_rent_return_map.html")

# 결과 지도 출력
m


In [ ]:
# ####### 출발지 -> 도착지 투명도 적용 - 너무 느림

# import folium
# import pandas as pd
# import numpy as np

# # 예시 데이터프레임 준비: 출발지 위도, 출발지 경도, 도착지 위도 경도, 사용량(가중치)
# grouped_df = grouped_df.dropna()
# grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)

# data = {
#     'start_lat': grouped_df['대여대여소위도'],
#     'start_lon': grouped_df['대여대여소경도'],
#     'end_lat': grouped_df['반납대여소위도'],
#     'end_lon': grouped_df['반납대여소경도'],
#     'usage': grouped_df['이용량']  # 가중치
# }
# df = pd.DataFrame(data)

# # Min-Max Scaling 적용
# df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
# df = df.sort_values(by='usage_scaled', ascending=True)

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 출발지-도착지 선 추가
# num_segments = 20  # 그라데이션 효과를 위해 분할할 구간 수
# for _, row in df.iterrows():
#     # 각 segment의 좌표 계산
#     latitudes = np.linspace(row['start_lat'], row['end_lat'], num_segments + 1)
#     longitudes = np.linspace(row['start_lon'], row['end_lon'], num_segments + 1)
    
#     # 가중치에 따른 색상 설정
#     weight = 2 + 12 * row['usage_scaled']  # 기본 두께 2, 최대 두께 14
#     red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 사용량에 비례
#     blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 사용량에 반비례
#     color = f'#{red_intensity:02x}55{blue_intensity:02x}'

#     # 각 segment마다 선을 그려서 점진적인 투명도 적용
#     for i in range(num_segments):
#         opacity = 0.8 * (1 - (i / num_segments))  # 점진적으로 투명도 감소
#         segment = [(latitudes[i], longitudes[i]), (latitudes[i + 1], longitudes[i + 1])]
#         folium.PolyLine(
#             segment,
#             color=color,
#             weight=weight,
#             opacity=opacity,
#             smooth_factor=2.0
#         ).add_to(m)

# # 결과 지도 저장
# m.save("bike_rent_return_map_transparent.html")


# # 결과 지도 출력
# m


In [ ]:
# import folium
# import pandas as pd

# # 예시 데이터프레임: 출발지 위도, 출발지 경도, 도착지 위도, 도착지 경도, 가중치(사용량)
# grouped_df = grouped_df.dropna()
# grouped_df = grouped_df.sort_values(by='이용량', ascending=False).head(10000)


# data = {
#     'start_lat': grouped_df['대여대여소위도'],
#     'start_lon': grouped_df['대여대여소경도'],
#     'end_lat': grouped_df['반납대여소위도'],
#     'end_lon': grouped_df['반납대여소경도'],
#     'usage': grouped_df['이용량']  # 가중치
# }

# # 데이터프레임 생성
# df = pd.DataFrame(data)

# # Min-Max Scaling 적용
# df['usage_scaled'] = (df['usage'] - df['usage'].min()) / (df['usage'].max() - df['usage'].min())

# # 사용량이 많은 순으로 정렬하여 높은 사용량의 선이 위에 그려지도록 조정
# df = df.sort_values(by='usage_scaled', ascending=True)

# # Folium 지도 생성 (서울을 중심으로 설정)
# m = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# # 출발지-도착지 선 추가
# for _, row in df.iterrows():
#     # 좌표 리스트 정의
#     locations = [(row['start_lat'], row['start_lon']), (row['end_lat'], row['end_lon'])]
    
#     # 가중치에 따른 밝기 및 선의 두께 설정
#     weight = 2 + 12 * row['usage_scaled']  # 기본 두께 2, 최대 두께 14
#     red_intensity = int(255 * row['usage_scaled'])  # 빨간색 강도는 이용량에 비례
#     blue_intensity = int(255 * (1 - row['usage_scaled']))  # 파란색 강도는 이용량에 반비례
#     color = f'#{red_intensity:02x}00{blue_intensity:02x}'  # 적은 쪽 파란색, 많은 쪽 빨간색
    
#     # PolyLine 추가 (빛나는 선 모양)
#     folium.PolyLine(
#         locations,
#         color=color,
#         weight=weight,
#         opacity=0.7,
#         tooltip=f'Usage: {row["usage"]}'
#     ).add_to(m)

# # 결과 지도 출력
# m
